<a href="https://colab.research.google.com/github/margaretxie/Test_Paper3_with_UCO/blob/main/UCO_v14_Manual_selected_selection2_with_obo_mea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test paper 3 - the HK one - with UCO dataset (only cam234, not cam 0 1, total 1296 videos )

# Paper 3 - https://link.springer.com/article/10.1007/s13755-023-00258-3  -skeleton based repeptition count method
# The paper's code - webpage: https://github.com/YuChengHSU/repetition-counting/tree/main

# UCO dataset [paper20] - downloaded from  http://rabinf24.uco.es/avaphy/    (Outlook Emailed got access )

# v14: v13_based, testing action 16 with selection2KPs, with obo & mae
# v13: v9_v10_v11 based, for testing action 16 with hand picked KPs, with final obo & mae

# v9: v6_based,for testing KPselection- only 81 videos of action 16 "a16", and only with KPs from 13-22 (10 KPs)

# v8: v4_based, 33KPs_obo, mea by type
# v7: v3_based, 33KPs_ merge_estimated_n_groundtrue_count
# v6: v2_based, apply original paper3 model (using whole 33 KPs)
# v5: average KP used, popular KPs analyst
# v4: calculate obo, mea according to action types (1-16)
# v3: merge_estimated_n_groundtrue_count
# v2: apply paper3's model to filtered KPs which visibility >=threshold
# v1: extract each videos' KPs  (this code only for half cameras (2,3,4))
# v0: framework of looping all the .mp4 files, with name PnAn (Person*Action*)#



# Setup
v="V12"

# KP used - 12, 14,16,24, 26, 28


# Import necessary libraries
from google.colab import drive
import pandas as pd
import numpy as np
import os
import subprocess
from datetime import datetime
import pytz

# Mount Google Drive
drive.mount('/content/drive')

results = []

base_path = '/content/drive/My Drive/repetition_dataset/UCO/extracedKP_npy_cam234'
KP33_xyz_path = '/content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz'
save_path = '/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only'

output_csv_path_2 = os.path.join(save_path, f"{v}_Cam234_output_a16_handpickKP.csv")


f=25 # frame rate


def get_modified_filename(filename):
    # Find the position of the last underscore
    last_underscore_index = filename.rfind('_')
    # Extract the part of the filename after the last underscore
    if last_underscore_index != -1:
        new_filename = filename[last_underscore_index + 1:]
    else:
        new_filename = filename
    return new_filename



# Ensure the output directory exists
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Function to process videos
def process_videos(base_path, kp_indices):
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if 'a16' in file:
                video_name = get_modified_filename(file)
                video_name = video_name.replace('.mp4', '.npy')
              #  print(f"Processing video: {video_name}")
                file_path = os.path.join(root, file)
                print(f"Processing {file_path}")
                data = np.load(file_path)

                # Use only KP 13-22
                filtered_data = data[:, kp_indices, :]

                # Save filtered data to a temporary .npy file
                temp_output_path = os.path.join(save_path, 'temp_filtered_data.npy')
                np.save(temp_output_path, filtered_data)

                # Prepare the command to run the external Python script with the necessary arguments
                command = [
                    "python", "rep-count.py",
                    "--data", temp_output_path,
                    "-j", str(len(kp_indices)),
                    "-d", "3",  # Only three dimensions are considered (x, y, z)
                    "-f", str(f),
                    "--wins", "256",
                    "--noverlap", "1"
                ]

                # Execute the command and capture the output
                try:
                    result = subprocess.run(command, check=True, capture_output=True, text=True)
                    output = result.stdout.strip()
                    estimated_count = int(output.split()[-1])  # Parse the count from the output
                    print("Output:", output)
                except subprocess.CalledProcessError as e:
                    print("Failed to execute command:", e)
                    print("Error output:", e.stderr)
                    estimated_count = None

                # Append results to the list
                results.append([video_name, len(kp_indices), estimated_count, data.shape[0]])

# KP - hand picked
kp_indices = [12, 14, 16, 24, 26, 28]

process_videos(KP33_xyz_path, kp_indices)

# Save the estimated counts of the filtered videos
current_time_str = datetime.now(pytz.timezone('America/New_York')).strftime('%Y%m%d%I%M%p').lower()
file_name = f"{v}_Cam234_output_{current_time_str}.csv"
output_csv_path = os.path.join(save_path, file_name)

results_df = pd.DataFrame(results, columns=['Video Name', 'KP Numbers', 'Estimated Count', 'Frame Number'])
results_df.to_csv(output_csv_path, index=False)

print("\n########First five rows of results_df:\n")
print(results_df.head(5))

print("\n########Last five rows of results_df:\n")
print(results_df.tail(5))

print(f"\nResults saved to {output_csv_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p12a16c2.npy
Output: Namespace(data='/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/temp_filtered_data.npy', j=6, d=3, f=25, wins=256, noverlap=1)
Estimated count 18
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p12a16c3.npy
Output: Namespace(data='/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/temp_filtered_data.npy', j=6, d=3, f=25, wins=256, noverlap=1)
Estimated count 5
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p12a16c4.npy
Output: Namespace(data='/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/temp_filtered_data.npy', j=6, d=3, f=25, wins=256, noverlap=1)
Estimated count 18
Processing /content/drive/My Drive/rep

In [ ]:
df = pd.read_csv(output_csv_path)

# Rename the 'Video Name' column to 'Name'
df.rename(columns={'Video Name': 'Name'}, inplace=True)

# Add a new column 'count' with all values set to 4 (UCO dataset, all the actions were taken 4 times)
df['count'] = 4

# Add a new column 'type' with the two numbers after the letter 'a' in the 'Name' values
df['type'] = df['Name'].apply(lambda x: x.split('a')[1][:2])


# Save the updated DataFrame to the new CSV file
df.to_csv(output_csv_path_2, index=False)


print(f'\n the combined estimated and groundtruth counts file has been save to {output_csv_path}')






 the combined estimated and groundtruth counts file has been save to /content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/V12_Cam234_output_202406190216pm.csv


In [ ]:



file_path = output_csv_path_2
new_file_path = os.path.join(base_path, f'V{v}_OBO_MAE_handpicked_KP_a16.csv' )

# Load the data
data = pd.read_csv(file_path)

# calculate OBO Correct Classification Rate
def calculate_obo(data):
    # Calculate OBO as the rate of correct classifications
    obo_correct = (abs(data['Estimated Count'] - data['count']) <= 1).mean()
    return obo_correct  # Higher is better (correct classification rate)

# Function to calculate MAE
def calculate_mae(data):
    conditions = data['count'] != 0
    data.loc[conditions, 'MAE'] = abs(data['Estimated Count'] - data['count']) / data['count']
    data.loc[~conditions, 'MAE'] = np.nan  # Treat division by zero cases as NaN
    return data['MAE'].mean(skipna=True)

# Calculating metrics for each type and overall
results = []
for type_ in data['type'].unique():
    type_data = data[data['type'] == type_].copy()
    obo_correct = calculate_obo(type_data)
    mae = calculate_mae(type_data)
    record_count = type_data.shape[0]
    results.append({'Type': type_, 'OBO (Higher Better)': round(obo_correct, 2), 'MAE (Lower Better)': round(mae, 2), 'Record Count': record_count})

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Calculate overall metrics
overall_obo = calculate_obo(data)
overall_mae = calculate_mae(data)

# Append overall metrics to the DataFrame using pd.concat
overall_df = pd.DataFrame([{'Type': 'Overall', 'OBO (Higher Better)': round(overall_obo, 2), 'MAE (Lower Better)': round(overall_mae, 2), 'Record Count': data.shape[0]}])
results_df = pd.concat([results_df, overall_df], ignore_index=True)

# Sort the DataFrame by OBO Correct Rate in ascending order
results_df = results_df.sort_values('OBO (Higher Better)', ascending=True)

# Print the sorted DataFrame before saving

print('')
print('')
print('     V13_UCO_Paper3_CAM234_OBO_MEA_type"A16"_handpicked_KPs  f=25')
print('   ===============================================================')
print('')
print(results_df)

# Save to new CSV file
results_df.to_csv(new_file_path, index=False)




     V13_UCO_Paper3_CAM234_OBO_MEA_type"A16"_handpicked_KPs  f=25

      Type  OBO (Higher Better)  MAE (Lower Better)  Record Count
0       16                 0.25                1.22            81
1  Overall                 0.25                1.22            81
